In [21]:
import numpy as np
from scipy import optimize

In [22]:
import pandas as pd

def load_csv(filename):
    df=np.genfromtxt(filename, delimiter=',')
    return df

# Find the min and max values for each column
def dataset_minmax(x,y):
    x=x/np.amax(x,axis=0)
    y=y/2
    return x,y


In [23]:
e=load_csv('cdnumeric.csv')

print(e.shape)

c=e[:,0:24]
b=e[:,[24]]
print(c.shape)
print(b.shape)

c,b=dataset_minmax(c,b)

(1000, 25)
(1000, 24)
(1000, 1)


In [24]:
c_train=c[:750]
c_test=c[750:]
b_train=b[:750]
b_test=b[750:]




In [25]:
class trainer(object):
    def __init__(self, N):
        #Make Local reference to network:
        self.N = N
    
    def costFunctionWrapper(self, params, x, y):
        self.N.setParams(params)
        cost = self.N.cost(x, y)
        grad = self.N.computeGradients(x,y)
        
        return cost, grad
        
    def train(self, x, y):
        #Make an internal variable for the callback function:
        self.x = x
        self.y = y
        
        params0 = self.N.getParams()

        options = {'maxiter': 200, 'disp' : True}
        _res = optimize.minimize(self.costFunctionWrapper, params0, jac=True, method='BFGS', \
                                 args=(x, y), options=options)

        self.N.setParams(_res.x)
        self.optimizationResults = _res

In [26]:
class Neural(object):
   
    #init is a constructor and self is the instance of the object created 
    #to sense that for which object the attributes belong to(self)
    def __init__(self,Lambda=0):
        self.noofInput=24
        self.noofOutput=1
        self.noofHidden=40
        self.Lambda = Lambda

        #rando.rndm generates arandom values with mean=0 and varience=1 by guassian distribution
        #parameters are the size of the matrix
        self.w1=np.random.randn(self.noofInput,self.noofHidden)
        self.w2=np.random.randn(self.noofHidden,self.noofOutput)
        
        #propagate input through the network
    def forward(self,x):
        #dot function for matrix multiplication
        self.z2=np.dot(x,self.w1)#adding up weights to input
        self.a2=self.sigmoid(self.z2)#activation function to scale the value of z1
        self.z3=np.dot(self.a2,self.w2)
        yhat=self.sigmoid(self.z3)
        return yhat
    
   #sigmoid is a activation function 
    def sigmoid(self,z):
        return 1/(1+np.exp(-z))
    #used in the propagation of cost function
    def sigmoidprime(self,z):
        return np.exp(-z)/(1+np.exp(-z))**2
    
    
    
    #difference to ouput and expected values i.e cost 
    def cost(self,x,y):
        self.yhat=self.forward(x)
        #regularization
        #adding sum of square of our weights to cost function
        cost1=0.5*sum((y-self.yhat)**2)/x.shape[0] + (self.Lambda/2)*(np.sum(self.w1**2)+np.sum(self.w2**2))
        return cost1
    
    #using gradient descent of backpropagation computing the cost minimum
    #i.e derivative of j(cost) by the weights w1 and w2
    def costminimum(self,x,y):
        self.yhat=self.forward(x)
        delta3=np.multiply(-(y-self.yhat),self.sigmoidprime(self.z3))
        #regularization
        djw2=np.dot(self.a2.T,delta3)/x.shape[0] + self.Lambda*self.w2
        
        delta2=np.dot(delta3,self.w2.T)*self.sigmoidprime(self.z2)
        #regularization
        djw1=np.dot(x.T,delta2)/x.shape[0] + self.Lambda*self.w1
        
        return djw1,djw2
    
    #NUMERICAL CHECKINGH
       #Helper Functions for interacting with other classes:
    def getParams(self):
        #Get W1 and W2 unrolled into vector:
        params = np.concatenate((self.w1.ravel(), self.w2.ravel()))
        return params
    
    def setParams(self, params):
        #Set W1 and W2 using single paramater vector.
        W1_start = 0
        W1_end = self.noofInput * self.noofHidden
        self.w1 = np.reshape(params[W1_start:W1_end], (self.noofInput , self.noofHidden))
        W2_end = W1_end + self.noofHidden*self.noofOutput
        self.w2 = np.reshape(params[W1_end:W2_end], (self.noofHidden, self.noofOutput))
        
    def computeGradients(self, x, y):
        dJdW1, dJdW2 = self.costminimum(x, y)
        return np.concatenate((dJdW1.ravel(), dJdW2.ravel()))
    
    
    

    
        

In [27]:
def computeNumericalGradient(N, x, y):
        paramsInitial = N.getParams()
        numgrad = np.zeros(paramsInitial.shape)
        perturb = np.zeros(paramsInitial.shape)
        e = 1e-4

        for p in range(len(paramsInitial)):
            #Set perturbation vector
            perturb[p] = e
            N.setParams(paramsInitial + perturb)
            loss2 = N.cost(x, y)
            
            N.setParams(paramsInitial - perturb)
            loss1 = N.cost(x, y)

            #Compute Numerical Gradient
            numgrad[p] = (loss2 - loss1) / (2*e)

            #Return the value we changed to zero:
            perturb[p] = 0
            
        #Return Params to original value:
        N.setParams(paramsInitial)

        return numgrad

In [28]:
nn=Neural(Lambda=0.0001)

#Make sure our gradients our correct after making changes:
numgrad = computeNumericalGradient(nn,c_train,b_train)
grad = nn.computeGradients(c_train,b_train)

#Should be less than 1e-8:
#norm(grad-numgrad)/norm(grad+numgrad)
np.linalg.norm(grad-numgrad)/np.linalg.norm(grad+numgrad)

1.2002307337347683e-10

In [29]:
nn=Neural()
T = trainer(nn)
T.train(c_train,b_train)

         Current function value: 0.015118
         Iterations: 200
         Function evaluations: 214
         Gradient evaluations: 214


In [30]:

got=nn.forward(c_test)

In [31]:
#print(got)

In [32]:
i=0
bad=0
good=0
unmatched=0
for i in range(0,250):
    if(b_test[i]==1):
        if(got[i]>0.6):
            good=good+1
        else:
            unmatched=unmatched+1
    else:
        if(got[i]<=0.6):
            bad=bad+1
        else:
            unmatched=unmatched+1
        
        

In [33]:
print(good,bad,unmatched)

56 107 87


In [34]:
i=0
bad_o=0
good_o=0
for i in range(0,250):
    if(b_test[i]!=0.5):
        good_o=good_o+1
    else:
        bad_o=bad_o+1

In [35]:
print(good_o,bad_o)

77 173
